In [25]:
import sqlite3

con = sqlite3.connect("via_data.db")
cur = con.cursor()

# Create tbl_location

In [26]:
cur.execute('''
          CREATE TABLE tbl_location (
            LOCATION_ID INTEGER PRIMARY KEY,
            LOCATION text UNIQUE NOT NULL,
            DATE_CREATED datetime default current_timestamp)
          ''')

#LOCATION_CODE text UNIQUE NOT NULL,

# Create tbl_route

In [27]:
cur.execute('''
          CREATE TABLE tbl_route (
            ROUTE_ID INTEGER PRIMARY KEY,
            ROUTE_START INTEGER NOT NULL REFERENCES tbl_location(LOCATION_ID),
            ROUTE_END INTEGER NOT NULL REFERENCES tbl_location(LOCATION_ID),
            DATE_CREATED datetime default current_timestamp,
            CONSTRAINT unique_route UNIQUE (ROUTE_START, ROUTE_END))
          ''')

# Create tbl_route

In [28]:
cur.execute('''
          CREATE TABLE tbl_train (
            TRAIN_ID INTEGER PRIMARY KEY,
            TRAIN_NUMBER text NOT NULL,
            ROUTE INTEGER NOT NULL REFERENCES tbl_route(ROUTE_ID),
            DATE_CREATED datetime default current_timestamp,
            CONSTRAINT unique_route UNIQUE (TRAIN_NUMBER, ROUTE))
          ''')

# create tbl_stop

In [29]:
cur.execute('''
          CREATE TABLE tbl_stop (
            STOP_ID INTEGER PRIMARY KEY,
            TRAIN INTEGER NOT NULL REFERENCES tbl_train(TRAIN_ID),
            STOP_LOCATION INTEGER NOT NULL REFERENCES tbl_location(LOCATION_ID),
            PREVIOUS_STOP_LOCATION INTEGER NOT NULL REFERENCES tbl_location(LOCATION_ID),
            DATE_CREATED datetime default current_timestamp,
            CONSTRAINT unique_route UNIQUE (TRAIN, STOP_LOCATION, PREVIOUS_STOP_LOCATION))
          ''')

# create tbl_via_data

In [30]:
cur.execute('''
          CREATE TABLE tbl_via_data (
            DATA_ID INTEGER PRIMARY KEY,
            TRAIN_STOP INTEGER NOT NULL REFERENCES tbl_stop(STOP_ID),
            SCHEDULE_DATETIME DATETIME NOT NULL,
            ARRIVAL_DATETIME DATETIME NOT NULL,
            MINUTES_LATE INTEGER NOT NULL,
            ETA text NOT NULL,
            DATE_TRAIN DATETIME NOT NULL,
            DATE_CREATED DATETIME default current_timestamp,
            CONSTRAINT unique_route UNIQUE (TRAIN_STOP, DATE_TRAIN))
          ''')

# Code for diagram

// Use DBML to define your database structure
// Docs: https://dbml.dbdiagram.io/docs

TABLE tbl_location {
  LOCATION_ID INTEGER [PRIMARY KEY]
  LOCATION text UNIQUE
  DATE_CREATED datetime
}

TABLE tbl_route {
  ROUTE_ID INTEGER [PRIMARY KEY]
  ROUTE_START INTEGER [ref: > tbl_location.LOCATION_ID]
  ROUTE_END INTEGER [ref: > tbl_location.LOCATION_ID]
  DATE_CREATED datetime
}

TABLE tbl_train{
  TRAIN_ID INTEGER [PRIMARY KEY]
  TRAIN_NUMBER text
  ROUTE INTEGER [ref: > tbl_route.ROUTE_ID]
  DATE_CREATED datetime
}

TABLE tbl_stop{
  STOP_ID INTEGER [PRIMARY KEY]
  TRAIN INTEGER [ref: > tbl_train.TRAIN_ID]
  STOP_LOCATION INTEGER [ref: > tbl_location.LOCATION_ID]
  PREVIOUS_STOP_LOCATION INTEGER [ref: > tbl_location.LOCATION_ID]
  DATE_CREATED datetime
}

TABLE tbl_via_data{
  DATA_ID INTEGER [PRIMARY KEY]
  TRAIN_STOP INTEGER [ref: > tbl_stop.STOP_ID]
  SCHEDULE_DATETIME DATETIME 
  ARRIVAL_DATETIME DATETIME 
  MINUTES_LATE INTEGER
  ETA text
  DATE_TRAIN datetime
  DATE_CREATED DATETIME
}